# Imports

In [6]:
import os
os.chdir('/home/jupyter/ee_tree_counting/Models/SSD')

In [7]:
from config import (
    DEVICE, 
    NUM_CLASSES, 
    NUM_EPOCHS, 
    OUT_DIR,
    VISUALIZE_TRANSFORMED_IMAGES, 
    NUM_WORKERS,
    RESIZE_TO,
    VALID_DIR,
    TRAIN_DIR,
    WEIGHTS_PATH
)
from model import create_model
from custom_utils import (
    Averager, 
    SaveBestModel, 
    save_model, 
    save_loss_plot,
    save_mAP
)
from tqdm.auto import tqdm
from datasets import (
    create_train_dataset, 
    create_valid_dataset, 
    create_train_loader, 
    create_valid_loader
)
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torch.optim.lr_scheduler import StepLR

import torch
import matplotlib.pyplot as plt
import time
import os

plt.style.use('ggplot')

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [8]:
# Function for running training iterations.
def train(train_data_loader, model):
    print('Training')
    model.train()
    
     # initialize tqdm progress bar
    prog_bar = tqdm(train_data_loader, total=len(train_data_loader))
    
    for i, data in enumerate(prog_bar):
        optimizer.zero_grad()
        images, targets = data
        
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        train_loss_hist.send(loss_value)

        losses.backward()
        optimizer.step()
    
        # update the loss value beside the progress bar for each iteration
        prog_bar.set_description(desc=f"Loss: {loss_value:.4f}")
    return loss_value

In [9]:
def validate(valid_data_loader, model):
    print('Validating')
    model.eval()
    
    # Initialize tqdm progress bar.
    prog_bar = tqdm(valid_data_loader, total=len(valid_data_loader))
    target = []
    preds = []
    for i, data in enumerate(prog_bar):
        images, targets = data
        
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
        
        with torch.no_grad():
            outputs = model(images, targets)

        # For mAP calculation using Torchmetrics.
        #####################################
        for i in range(len(images)):
            true_dict = dict()
            preds_dict = dict()
            true_dict['boxes'] = targets[i]['boxes'].detach().cpu()
            true_dict['labels'] = targets[i]['labels'].detach().cpu()
            preds_dict['boxes'] = outputs[i]['boxes'].detach().cpu()
            preds_dict['scores'] = outputs[i]['scores'].detach().cpu()
            preds_dict['labels'] = outputs[i]['labels'].detach().cpu()
            preds.append(preds_dict)
            target.append(true_dict)
        #####################################

    metric = MeanAveragePrecision()
    metric.update(preds, target)
    metric_summary = metric.compute()
    return metric_summary

In [ ]:
os.makedirs('outputs', exist_ok=True)
train_dataset = create_train_dataset(TRAIN_DIR)
valid_dataset = create_valid_dataset(VALID_DIR)
train_loader = create_train_loader(train_dataset, NUM_WORKERS)
valid_loader = create_valid_loader(valid_dataset, NUM_WORKERS)
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(valid_dataset)}\n")

# Initialize the model and move to the computation device.
model = create_model(num_classes=NUM_CLASSES, size=RESIZE_TO, weights_path=WEIGHTS_PATH)
model = model.to(DEVICE)
# print(model)

# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.0001, momentum=0.9, nesterov=True)
scheduler = StepLR(
    optimizer=optimizer, step_size=15, gamma=0.1, verbose=True
)

# To monitor training loss
train_loss_hist = Averager()
# To store training loss and mAP values.
train_loss_list = []
map_50_list = []
map_list = []

# Mame to save the trained model with.
MODEL_NAME = 'model'

# Whether to show transformed images from data loader or not.
if VISUALIZE_TRANSFORMED_IMAGES:
    from custom_utils import show_tranformed_image
    show_tranformed_image(train_loader)

# To save best model.
save_best_model = SaveBestModel()

# Training loop.
for epoch in range(NUM_EPOCHS):
    print(f"\nEPOCH {epoch+1} of {NUM_EPOCHS}")

    # Reset the training loss histories for the current epoch.
    train_loss_hist.reset()

    # Start timer and carry out training and validation.
    start = time.time()
    train_loss = train(train_loader, model)
    metric_summary = validate(valid_loader, model)
    print(f"Epoch #{epoch+1} train loss: {train_loss_hist.value:.3f}")   
    print(f"Epoch #{epoch+1} mAP: {metric_summary['map']}")   
    end = time.time()
    print(f"Took {((end - start) / 60):.3f} minutes for epoch {epoch}")

    train_loss_list.append(train_loss)
    map_50_list.append(metric_summary['map_50'])
    map_list.append(metric_summary['map'])

    # save the best model till now.
    save_best_model(
        model, float(metric_summary['map']), epoch, 'outputs'
    )
    # Save the current epoch model.
    save_model(epoch, model, optimizer)

    # Save loss plot.
    save_loss_plot(OUT_DIR, train_loss_list)

    # Save mAP plot.
    save_mAP(OUT_DIR, map_50_list, map_list)
    scheduler.step()

# Model Evaluation

In [6]:
model = create_model(num_classes=NUM_CLASSES, size=640, weights_path=WEIGHTS_PATH)
checkpoint = torch.load('outputs/best_model.pth', map_location=DEVICE)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE).eval()

test_dataset = create_valid_dataset(
    '/home/jupyter/ee_tree_counting/Data/Combined Dataset XML No Aug/test'
)
test_loader = create_valid_loader(test_dataset, num_workers=NUM_WORKERS)

metric_summary = validate(test_loader, model)
print(f"mAP_50: {metric_summary['map_50']*100:.3f}")
print(f"mAP_50_95: {metric_summary['map']*100:.3f}")

Validating


  0%|          | 0/12 [00:00<?, ?it/s]

mAP_50: 65.223
mAP_50_95: 25.701


# Exporting to CSV file format

In [21]:
import pandas as pd

# Create a dictionary with the data
data = {
    "Epoch": list(range(1, 101)),
    "Train Loss": [
        4.938, 4.075, 3.83, 3.696, 3.61, 3.531, 3.463, 3.412, 3.359, 3.322, 3.264, 3.22, 3.179, 3.136, 3.104, 3.063,
        3.073, 3.057, 3.058, 3.058, 3.049, 3.041, 3.036, 3.03, 3.04, 3.034, 3.017, 3.03, 3.026, 3.014, 3.018, 3.014,
        3.012, 3.017, 3.005, 3.008, 3.015, 3.009, 3.008, 3.003, 3.009, 2.999, 3.02, 3.014, 3.011, 3.007, 3.009, 3.006,
        3.005, 3.008, 3.013, 3.011, 3.004, 3.007, 3.013, 3.012, 3.007, 3.007, 3.003, 3.002, 3.014, 3.014, 3.012, 3.01,
        3.007, 3.009, 3.006, 3.004, 3.011, 3.001, 3.012, 3.01, 3.007, 3.015, 3.009, 3.004, 3.013, 3.007, 3.004, 3.01,
        3.012, 3.007, 3.009, 3.009, 3.007, 3.017, 3.01, 3.004, 3.013, 3.007, 3.014, 3.009, 3.006, 3.007, 3.008, 3.01,
        3.008, 3.007, 3.006, 3.008
    ],
    "mAP": [
        0.0020383174996823072, 0.022861845791339874, 0.08308479189872742, 0.14638210833072662, 0.18756431341171265,
        0.20833797752857208, 0.2192489504814148, 0.22915536165237427, 0.23228442668914795, 0.24335764348506927,
        0.24488072097301483, 0.24967779219150543, 0.25477665662765503, 0.2594020962715149, 0.2659323215484619,
        0.262830913066864, 0.26334813237190247, 0.26300138235092163, 0.2632383406162262, 0.2628127336502075,
        0.2647073566913605, 0.2639809250831604, 0.2655472755432129, 0.2646346390247345, 0.2666299641132355,
        0.2656112015247345, 0.2669174373149872, 0.26502883434295654, 0.2670244872570038, 0.26789408922195435,
        0.2672414183616638, 0.26779478788375854, 0.2676714062690735, 0.26808544993400574, 0.2677479684352875,
        0.2680380940437317, 0.26814815402030945, 0.2681480348110199, 0.2679618000984192, 0.2680222690105438,
        0.2683304250240326, 0.2683519721031189, 0.2682887315750122, 0.2683258354663849, 0.2684342861175537,
        0.2684059739112854, 0.2683124840259552, 0.2683514654636383, 0.26834672689437866, 0.2683456838130951,
        0.26834726333618164, 0.2683553993701935, 0.26837506890296936, 0.26835018396377563, 0.26836878061294556,
        0.2683883309364319, 0.268428236246109, 0.2684257924556732, 0.2681969106197357, 0.2683669328689575,
        0.2683667540550232, 0.2683667540550232, 0.2683671712875366, 0.2683606445789337, 0.26836055517196655,
        0.2683664560317993, 0.26836705207824707, 0.2683640420436859, 0.26836854219436646, 0.2683689296245575,
        0.2683689296245575, 0.2683689296245575, 0.26836901903152466, 0.2683681845664978, 0.2683696448802948,
        0.2683696448802948, 0.2683696448802948, 0.2683698832988739, 0.2683696448802948, 0.2683698832988739,
        0.2683698832988739, 0.2683698832988739, 0.2683698832988739, 0.2683698832988739, 0.2683698832988739,
        0.2683698832988739, 0.2683698832988739, 0.2683698832988739, 0.2683698832988739, 0.2683698832988739,
        0.2683698832988739, 0.2683698832988739, 0.2683698832988739, 0.2683698832988739, 0.2683698832988739,
        0.26836860179901123, 0.2683698832988739, 0.2683698832988739, 0.2683698832988739, 0.2683698832988739
    ]
}

# Convert dictionary to DataFrame
df = pd.DataFrame(data)

# Save DataFrame to CSV

df.to_csv('training_log_100eNoAug.csv', index=False)

csv_file_path


'outputs'